In [ ]:
        ## CS 230 PROJECT BY ELLEN ROPER (eroper) AND TRISTAN GOSAKTI (tgosakti) ##
import math
import numpy as np
import h5py
import matplotlib.pyplot as plt
import tensorflow as tf
from tensorflow.python.framework import ops
import json
import matplotlib.pyplot as plt

strokeTypes = 5
fullSize = 2000
%matplotlib inline


def shuffle_in_unison(a, b):
    rng_state = np.random.get_state()
    np.random.shuffle(a)
    np.random.set_state(rng_state)
    np.random.shuffle(b)

In [ ]:
### DATA PARSING ###
trainSamples = []
trainSamplesCNN = []
groundTruth = []
noisy = []
noisycnn = []
noisytruth = []
noisytruthcnn = []
groundtruthcnn = []

for strokeType in range (1, strokeTypes+1):
    noFiles = 10
    for i in range (0, noFiles): #processing every file/batch #FIX LATER

        #naming
        frameDataName = str(strokeType) + "-" + str(i+1) + "-TSFrameData.txt"
        motionDataName = str(strokeType) + "-" + str(i+1) + "-TSMotionData.txt"

        #loading and processing
        frameData = np.loadtxt(frameDataName)
            
        with open(motionDataName) as f: #for every file
            for line in f: #for every json entry, should be around 10 
                line = json.loads(line)
                oneSample = []
                for i in range (int(line["startIndex"]), int(line["endIndex"])):
                    
                    
                    timeSlice = frameData[i]
                    timeSlice = np.delete(timeSlice[1:], slice(None, None, 4))

                    oneSample.append(timeSlice)
                oneSampleNp = np.vstack(oneSample)
                oneSampleFlatten = oneSampleNp.flatten()
                noise = np.random.normal(0.0, 1, oneSampleNp.shape)
                flattenNoise = noise.flatten()
                oneSampleFlattenup = oneSampleFlatten + flattenNoise
                oneSampleFlattendown = oneSampleFlatten - flattenNoise

                
                #TRAIN DATA FOR CNN
                CNNSample = np.transpose(oneSampleNp)
                CNNSampleup = CNNSample+np.transpose(noise)
                CNNSampledown = CNNSample-np.transpose(noise)
                CNNSamplepadded = np.pad(CNNSample, [(0,0), (0,111-CNNSample.shape[1])], 'constant')
                CNNSampleuppadded = np.pad(CNNSampleup,[(0,0), (0,111-CNNSampleup.shape[1])], 'constant')
                CNNSampledownpadded = np.pad(CNNSampledown, [(0,0), (0,111-CNNSampledown.shape[1])], 'constant')
                trainSamplesCNN.append(CNNSamplepadded)
                noisycnn.append(CNNSampleuppadded)
                noisycnn.append(CNNSampledownpadded)
                
                #TRAIN DATA FOR OTHER MODELS
                oneSamplePadded = np.pad(oneSampleFlatten, (0, fullSize-oneSampleFlatten.size), 'constant')             
                oneSampleFlattenupPadded = np.pad(oneSampleFlattenup, (0, fullSize-oneSampleFlattenup.size), 'constant')
                oneSampleFlattendownPadded = np.pad(oneSampleFlattendown, (0, fullSize-oneSampleFlattendown.size), 'constant')
                noisy.append(oneSampleFlattenupPadded)
                noisy.append(oneSampleFlattendownPadded)
                trainSamples.append(oneSamplePadded)
                
                
                #making y for every x
                singleY = [0] * strokeTypes
                singleY[strokeType-1] = 1
                groundTruth.append(singleY)
                groundtruthcnn.append(singleY)
                noisytruthcnn.append(singleY)
                noisytruthcnn.append(singleY)
                noisytruth.append(singleY)
                noisytruth.append(singleY)

x_train_cnn1 = np.asarray(trainSamplesCNN)
y_train_cnn1 = np.asarray(groundtruthcnn)

noisy_train_cnn = np.asarray(noisycnn)
noisy_train_y_cnn = np.asarray(noisytruthcnn)

trainSamplesnp = np.vstack(trainSamples)
groundTruthnp = np.vstack(groundTruth)
noisySamplesnp = np.vstack(noisy)
noisytruthnp = np.vstack(noisytruth)

x_train1 = np.transpose(trainSamplesnp)
y_train1 = np.transpose(groundTruthnp)
noisy_train = np.transpose(noisySamplesnp)
noisy_train_y = np.transpose(noisytruthnp)
    
shuffle_in_unison(np.transpose(x_train1), np.transpose(y_train1))
shuffle_in_unison(x_train_cnn1, y_train_cnn1)

x_train_cnn = x_train_cnn1[0:407]
y_train_cnn = y_train_cnn1[0:407]
x_dev_cnn = x_train_cnn1[407:457]
y_dev_cnn = y_train_cnn1[407:457]
x_test_cnn = x_train_cnn1[457:507]
y_test_cnn = y_train_cnn1[457:507]

x_dev_cnn = np.expand_dims(x_dev_cnn, axis=3)
x_test_cnn = np.expand_dims(x_test_cnn, axis=3)

alltrain = []
alltrainy = []
for k in range(0,407):
    alltrain.append(x_train_cnn[k])
    alltrainy.append(y_train_cnn[k])
for l in range(0, 1014):
    alltrain.append(noisy_train_cnn[l])
    alltrainy.append(noisy_train_y_cnn[l])
what = np.asarray(alltrain)
y_train_cnn = np.asarray(alltrainy)
x_train_cnn = np.expand_dims(what, axis=3)

x_trainT = np.transpose(x_train1)
y_trainT = np.transpose(y_train1)

x_train = np.transpose(x_trainT[:407])
y_train = np.transpose(y_trainT[:407])
x_dev = np.transpose(x_trainT[407:457])
y_dev = np.transpose(y_trainT[407:457])
x_test = np.transpose(x_trainT[457:507])
y_test = np.transpose(y_trainT[457:507])

x_train = np.concatenate((x_train, noisy_train), axis=1)
y_train = np.concatenate((y_train, noisy_train_y), axis=1)
shuffle_in_unison(np.transpose(x_train), np.transpose(y_train))

In [ ]:
### CNN ###
def create_placeholders(height_input, width_input, num_channels, num_strokes):
    X = tf.placeholder(tf.float32, [None, height_input, width_input, num_channels])
    Y = tf.placeholder(tf.float32, [None, num_strokes])
    return X,Y

def initialize_parameters():
    tf.set_random_seed(1)
    w1 = tf.get_variable("w1", [3,3,1,10], initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    w2 = tf.get_variable("w2", [6,5,10,10], initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    w3 = tf.get_variable("w3", [1,5,10,1], initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    parameters = {"w1": w1, "w2": w2, "w3": w3}
    return parameters

def forward_propagation(X, parameters):
    w1 = parameters['w1']
    w2 = parameters['w2']
    w3 = parameters['w3']
    
    Z1 = tf.nn.conv2d(X, w1, strides = [1,3,1,1], padding = 'VALID') 
    A1 = tf.nn.relu(Z1)
    Z2 = tf.nn.conv2d(A1, w2, strides = [1,1,1,1], padding='VALID')
    A2 = tf.nn.relu(Z2)
    P2 = tf.nn.max_pool(A2, ksize = [1,1,2,1], strides = [1,1,2,1], padding = 'VALID')
    P2 = tf.contrib.layers.flatten(P2)
    Z4 = tf.contrib.layers.fully_connected(P2, 5, activation_fn =tf.nn.softmax)
    return Z4

def compute_cost(Z4, Y):
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=Z4, labels=Y))
    return cost
    

In [ ]:
### CNN Model ###
tf.reset_default_graph()
tf.set_random_seed(0)
(m, height, width, num_channels) = x_train_cnn.shape
n_y = y_train_cnn.shape[1]
costs = []
learning_rate = 0.001
X,Y = create_placeholders(height,width,1,n_y)
parameters = initialize_parameters()
Z4 = forward_propagation(X, parameters)
cost = compute_cost(Z4, Y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
init = tf.global_variables_initializer()

num_batches = m;
with tf.Session() as sess:
    sess.run(init)
    for i in range(55):
        sess.run([optimizer,cost], feed_dict={X: x_train_cnn, Y: y_train_cnn})
        
    parameters = sess.run(parameters)
    correct_prediction = tf.equal(tf.argmax(Z4,1), tf.argmax(Y,1))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train Accuracy:", accuracy.eval({X: x_train_cnn, Y: y_train_cnn}))
    print("Dev Accuracy: ", accuracy.eval({X: x_dev_cnn, Y: y_dev_cnn}))
    print("Test Accuracy:", accuracy.eval({X: x_test_cnn, Y: y_test_cnn}))
    labels = tf.argmax(Y, axis = 1)
    predicted = tf.argmax(Z4, axis = 1)
    confusion_matrix = tf.confusion_matrix(labels, predicted, 5) #5 strokes

In [ ]:
### NEURAL NETWORK IMPLEMENTATION: 1 hidden layer ###
#create placeholders: 
def create_placeholders(n_x, n_y):
    X = tf.placeholder(tf.float32, [n_x, None], name = "X")
    Y = tf.placeholder(tf.float32, [n_y, None], name = "Y")
    return X,Y

#initialize weight and biases
def initialize_parameters(n_x, n_y, n_hidden_1):
    w1 = tf.get_variable("w1", shape=(n_hidden_1, n_x), initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    w2 = tf.get_variable("w2", shape=(n_y, n_hidden_1), initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    b1 = tf.get_variable("b1", [n_hidden_1,1], initializer = tf.zeros_initializer())
    b2 = tf.get_variable("b2", [num_classes,1], initializer=tf.zeros_initializer())
    parameters = {"W1": w1, "W2": w2, "b1": b1, "b2": b2}
    return parameters


#create the model
def forward_propagation(x, parameters):
    W1 = parameters['W1']
    layer_1 = tf.matmul(W1,x) + parameters['b1']               
    out_layer = tf.matmul(parameters['W2'], layer_1) + parameters['b2']
    return out_layer

def compute_cost(Z, Y):
    logits = tf.transpose(Z)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    prediction = tf.nn.softmax(logits)
    return cost

In [ ]:
### 1 Hidden Layer ###
tf.reset_default_graph()
tf.set_random_seed(0)
X_train = x_train
Y_train = y_train
X_test = x_test
Y_test = y_test
learning_rate = 0.1
n_hidden_1 = 500
num_classes = strokeTypes
(n_x, m) = X_train.shape
n_y = Y_train.shape[0]
X,Y = create_placeholders(n_x, n_y)
parameters = initialize_parameters(n_x, n_y, n_hidden_1)
Z = forward_propagation(X, parameters) 
cost = compute_cost(Z, Y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)
init = tf.global_variables_initializer()

#start training
with tf.Session() as sess:
    sess.run(init)
    for i in range(10):
        sess.run([optimizer, cost], feed_dict={X: x_train, Y: y_train})
    parameters = sess.run(parameters)
    correct_prediction = tf.equal(tf.argmax(Z), tf.argmax(Y))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train Accuracy:", accuracy.eval({X: x_train, Y: y_train}))
    print("Dev accuracy: ", accuracy.eval({X: x_dev, Y: y_dev}))
    print("Test Accuracy:", accuracy.eval({X: x_test, Y: y_test}))

In [ ]:
### DEEP NETWORK IMPLEMENTATION ###
def create_placeholders_deep(n_x, n_y):
    X = tf.placeholder(tf.float32, shape=(n_x,None), name="X")
    Y = tf.placeholder(tf.float32, shape=(n_y,None), name="Y")
    return X,Y

def initialize_parameters_deep(unitsvector, n_x):    
    w1 = tf.get_variable("w1", shape=(unitsvector['n1'], n_x), initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    b1 = tf.get_variable("b1", [unitsvector['n1'], 1], initializer=tf.zeros_initializer())
    w2 = tf.get_variable("w2", shape=(unitsvector['n2'],unitsvector['n1']), initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    b2 = tf.get_variable("b2", [unitsvector['n2'],1], initializer=tf.zeros_initializer())
    w3 = tf.get_variable("w3", shape=(unitsvector['n3'], unitsvector['n2']), initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    b3 = tf.get_variable("b3", [unitsvector['n3'], 1], initializer=tf.zeros_initializer())
    w4 = tf.get_variable("w4", shape = (unitsvector['n4'], unitsvector['n3']), initializer=tf.contrib.layers.xavier_initializer(seed = 0))
    b4 = tf.get_variable("b4", [unitsvector['n4'], 1], initializer=tf.zeros_initializer())
    parameters = {"w1":w1, "w2":w2, "w3": w3, "w4": w4, "b1":b1, "b2":b2, "b3": b3, "b4": b4}
    return parameters

def forward_propagation_deep(X, parameters):
    w1 = parameters['w1']
    b1 = parameters['b1']
    w2 = parameters['w2']
    b2 = parameters['b2']
    w3 = parameters['w3']
    b3 = parameters['b3']
    w4 = parameters['w4']
    b4 = parameters['b4']
    
    Z1 = tf.add(tf.matmul(w1,X), b1)
    A1 = tf.nn.relu(Z1)
    Z2 = tf.add(tf.matmul(w2,A1), b2)
    A2 = tf.nn.relu(Z2)
    Z3 = tf.add(tf.matmul(w3, A2), b3)
    A3 = tf.nn.relu(Z3)
    Z4 = tf.add(tf.matmul(w4, A3), b4)
    
    return Z4

def compute_cost_deep(Z4, Y):
    logits = tf.transpose(Z4)
    labels = tf.transpose(Y)
    cost = tf.reduce_mean(tf.nn.softmax_cross_entropy_with_logits_v2(logits=logits, labels=labels))
    return cost

In [ ]:
### DEEP NETWORK ###
tf.reset_default_graph()
tf.set_random_seed(0)
X_train = x_train
Y_train= y_train
X_test = x_test
Y_test = y_test
learning_rate = 0.01
num1 = 550
num2 = 300
num3 = 100
num4 = 5
num_classes = 5
(n_x, m) = X_train.shape
n_y = Y_train.shape[0]
costs = []
X,Y = create_placeholders_deep(n_x,n_y)
unitsvector = {"n1": num1, "n2": num2, "n3": num3, "n4": num4}
parameters = initialize_parameters_deep(unitsvector, n_x)
Z4 = forward_propagation_deep(X, parameters)
cost = compute_cost_deep(Z4, Y)
optimizer = tf.train.AdamOptimizer(learning_rate=learning_rate).minimize(cost)

init = tf.global_variables_initializer()

with tf.Session() as sess:
    sess.run(init)
    for i in range(20):
        sess.run([optimizer,cost], feed_dict={X: X_train, Y:Y_train})
    parameters = sess.run(parameters)
    correct_prediction = tf.equal(tf.argmax(Z4), tf.argmax(Y))
    accuracy = tf.reduce_mean(tf.cast(correct_prediction, "float"))
    print("Train Accuracy: ", accuracy.eval({X: X_train, Y: Y_train}))
    print("Dev Accuracy: ", accuracy.eval({X: x_dev, Y: y_dev}))
    print("Test Accuracy: ", accuracy.eval({X: X_test, Y: Y_test}))